## Демонстрация пайплайна обработки PDF-документов

**Цель:** Показать полный цикл обработки PDF-файла - от извлечения текста через OCR до структурирования данных в JSON.

## Архитектура проекта

```
PDF → Конвертация в изображения → Предобработка → OCR → Извлечение данных → JSON
```
#### В телеграм-боте

```
Команда /start → Главное меню
↓
Кнопка "Скан" → Ожидание PDF
↓
[Telegram Bot]
↓ (PDF файл)
[FastAPI Server] -> проверка на формат
↓
PDF → Конвертация в изображения → Предобработка → OCR → Извлечение данных → JSON
↓
[Telegram Bot]
↓
Отображение результата пользователю
```

Установка необходимых библиотек

In [1]:
%pip install pymupdf pdfplumber opencv-python pillow matplotlib ipykernel python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.2 MB/s eta 0:00:00


Импорт библиотек

In [2]:
import fitz
import pdfplumber
import re
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

Создание структуры модулей

In [12]:
# modules/pdf_reader.py
def read_pdf_text(pdf_path: str) -> str:
    """
    Извлечение текста из PDF. Если PDF содержит таблицы — pdfplumber их тоже отдаст.
    """
    text = []

    # 1) pdfplumber
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text.append(page_text)
    except Exception:
        pass

    # 2) PyMuPDF
    if not text:
        doc = fitz.open(pdf_path)
        for page in doc:
            text.append(page.get_text())
        doc.close()

    return "\n".join(text)

# modules/preprocess.py
def clean_text(text: str) -> str:
    text = text.replace("\xa0", " ")
    text = re.sub(r"[ ]{2,}", " ", text)
    text = re.sub(r"\n{3,}", "\n", text)
    return text.strip()

def normalize(text: str) -> str:
    """
    Универсальная коррекция ошибок PDF/OCR.
    """
    corrections = {
        'Ng': '№',
        'н/н': '№',
        '0ОО': 'ООО',
        'цена,руб': 'цена руб',
        'руб:': 'руб.',
        'Ipy6': 'руб',
        'Oт': 'от',
    }

    for wrong, correct in corrections.items():
        text = text.replace(wrong, correct)

    return text

# modules/extract/general.py
def extract_document_type(text):
    t = text.lower()
    if "счет на оплату" in t or "счёт на оплату" in t:
        return "Счёт на оплату"
    if "счет-оферта" in t:
        return "Счёт-оферта"
    if "акт выполненных работ" in t:
        return "Акт выполненных работ"
    if "счет" in t or "счёт" in t:
        return "Счёт"
    return None

def extract_document_number(text):
    text = normalize(text)
    patterns = [
        r"сч[её]т\s*№\s*([A-Za-zА-Яа-я0-9\-\/]+)",
        r"№\s*([A-Za-zА-Яа-я0-9\-\/]+)\s*от",
    ]
    for p in patterns:
        m = re.search(p, text, re.IGNORECASE)
        if m:
            return m.group(1)
    return None

def extract_document_date(text):
    m = re.search(r"(\d{1,2}[.\-/]\d{1,2}[.\-/]\d{4})", text)
    if m:
        return m.group(1)
    return None

# modules/extract/addresses.py
ADDRESS_RE = re.compile(
    r"(?:\b\d{6},\s*)?"                        # индекс
    r"(?:[А-ЯЁа-яё][А-ЯЁа-яё\- ]+?,\s*){1,4}"  # регион/город
    r"(?:ул\.?|улица|проспект|пр-т|пер\.?|переулок|ш\.?|шоссе)\s+[А-ЯЁа-яё0-9\- ]+[, ]*"
    r"(?:д\.?|дом)\s*\d+[А-Яа-я0-9\-]*",
    re.IGNORECASE
)

def extract_address_block(text: str):
    m = ADDRESS_RE.search(text)
    if m:
        return m.group(0).strip().rstrip(",.")
    return None

# modules/extract/amounts.py
def clean_amount(v):
    if not v:
        return None
    v = re.sub(r"[^\d.,]", "", v)
    return v.replace(",", ".").strip()

def extract_amounts(text):
    total = re.search(r"(?:Итого|Всего)[^\d]+([\d\s.,]+)", text)
    vat = re.search(r"НДС[^\d]+([\d\s.,]+)", text)

    return {
        "total_amount": clean_amount(total.group(1)) if total else None,
        "vat_amount": clean_amount(vat.group(1)) if vat else None,
        "vat_rate": "20%" if vat else "Без НДС"
    }

# modules/extract/bank.py
def extract_bank_details(text):
    bik = re.search(r"БИК[: ]*(\d{9})", text)
    rs = re.search(r"(?:р/с|р\.с)[^\d]*(\d{20})", text)
    ks = re.search(r"(?:к/с|кор\.с)[^\d]*(\d{20})", text)

    return {
        "bik": bik.group(1) if bik else None,
        "account": rs.group(1) if rs else None,
        "correspondent_account": ks.group(1) if ks else None
    }

# modules/extract/blocks.py
BUYER_MARKERS = [
    "Покупатель", "Заказчик", "Грузополучатель",
    "Клиент", "Плательщик"
]

def split_supplier_buyer(text: str):
    pattern = "(" + "|".join(BUYER_MARKERS) + ")[:\\s]"
    parts = re.split(pattern, text, maxsplit=1, flags=re.IGNORECASE)

    if len(parts) < 3:
        return text, ""

    supplier = parts[0]
    buyer = parts[1] + parts[2]
    return supplier, buyer

# modules/extract/entities.py
def clean_org_name(name: str | None) -> str | None:
    if not name:
        return name

    name = re.sub(r"\bИНН[\s\d]*$", "", name)
    name = re.sub(r"\bКПП[\s\d]*$", "", name)
    name = re.sub(r"[ ,]+$", "", name)
    name = name.replace(" ,", ",")
    name = re.sub(r"\s{2,}", " ", name)
    return name.strip()

def extract_inn_kpp_block(text: str):
    inn = None
    kpp = None

    m = re.search(r"ИНН[:\s]*([0-9]{10,12})", text, re.IGNORECASE)
    if m:
        inn = m.group(1)

    m = re.search(r"КПП[:\s]*([0-9]{9})", text, re.IGNORECASE)
    if m:
        kpp = m.group(1)

    return inn, kpp

def extract_name_block(text: str, inn: str | None):
    if inn:
        # Берём строку ДО ИНН
        pattern = r"(.{3,100}?)\s*(?:ИНН\s*%s)" % inn
        m = re.search(pattern, text, flags=re.IGNORECASE)
        if m:
            return clean_org_name(m.group(1))

    # fallback — первая строчная строка с ООО/АО/ИП/ПАО
    m = re.search(
        r"(ООО|АО|ПАО|ИП|ЗАО|ОАО)[^\n,]+",
        text,
        re.IGNORECASE
    )
    return m.group(0).strip() if m else None

# modules/extract/items.py
def extract_items_from_pdf(pdf_path: str):
    """
    Универсальный парсер таблиц PDF
    """
    items = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                if not table:
                    continue

                header = table[0]
                rows = table[1:]

                if "№" not in header[0]:
                    continue

                for r in rows:
                    item = {
                        "name": None,
                        "qty": None,
                        "unit": None,
                        "price": None,
                        "total": None
                    }

                    for i, h in enumerate(header):
                        if not h:
                            continue

                        h_low = h.lower()

                        if "наим" in h_low or "товар" in h_low:
                            item["name"] = r[i]
                        elif "кол" in h_low:
                            item["qty"] = r[i]
                        elif "ед" in h_low:
                            item["unit"] = r[i]
                        elif "цен" in h_low:
                            item["price"] = r[i]
                        elif "сум" in h_low:
                            item["total"] = r[i]

                    items.append(item)

    return items

Основная функция обработки PDF

In [13]:
def process_pdf(pdf_path: str) -> dict:
    text = read_pdf_text(pdf_path)
    text = normalize(clean_text(text))

    supplier_block, buyer_block = split_supplier_buyer(text)

    # --- Поставщик ---
    supplier_inn, supplier_kpp = extract_inn_kpp_block(supplier_block)
    supplier_name = extract_name_block(supplier_block, supplier_inn)
    supplier_name = clean_org_name(supplier_name)
    supplier_addr = extract_address_block(supplier_block)

    # --- Покупатель ---
    buyer_inn, buyer_kpp = extract_inn_kpp_block(buyer_block)
    buyer_name = extract_name_block(buyer_block, buyer_inn)
    buyer_name = clean_org_name(buyer_name)
    buyer_addr = extract_address_block(buyer_block)

    # --- Банк (ищем только в блоке поставщика) ---
    bank_data = extract_bank_details(supplier_block)

    amounts = extract_amounts(text)
    items = extract_items_from_pdf(pdf_path)

    return {
        "document_type": extract_document_type(text),
        "document_number": extract_document_number(text),
        "document_date": extract_document_date(text),

        "supplier": {
            "name": supplier_name,
            "inn": supplier_inn,
            "kpp": supplier_kpp,
            "address": supplier_addr,
            **bank_data
        },

        "buyer": {
            "name": buyer_name,
            "inn": buyer_inn,
            "kpp": buyer_kpp,
            "address": buyer_addr
        },

        **amounts,
        "items": items,

        "payment_purpose": None,
        "payment_deadline": None
    }

Указание пути к PDF файлу

In [14]:
pdf_path = "/content/example1.pdf"

print(f"Используется PDF файл: {pdf_path}")

if not os.path.exists(pdf_path):
    print(f"Файл {pdf_path} не найден!")
    print("Пожалуйста, укажите правильный путь к файлу 'счет.pdf'")
else:
    print(f"Файл {pdf_path} найден")

Используется PDF файл: /content/example1.pdf
✅ Файл /content/example1.pdf найден, размер: 95505 байт


Демонстрация работы

In [15]:
if os.path.exists(pdf_path):
    print("Обработка PDF-документа...")

    try:
        result = process_pdf(pdf_path)

        print("\n  JSON-формат")
        print(json.dumps(result, ensure_ascii=False, indent=2))

        print("\n Обычный вывод")
        print(f"Тип документа: {result['document_type']}")
        print(f"Номер документа: {result['document_number']}")
        print(f"Дата документа: {result['document_date']}")

        print(f"\n--- Поставщик ---")
        print(f"Название: {result['supplier']['name']}")
        print(f"ИНН: {result['supplier']['inn']}")
        print(f"КПП: {result['supplier']['kpp']}")
        print(f"Адрес: {result['supplier']['address']}")
        print(f"БИК: {result['supplier']['bik']}")
        print(f"Расчетный счет: {result['supplier']['account']}")

        print(f"\n--- Покупатель ---")
        print(f"Название: {result['buyer']['name']}")
        print(f"ИНН: {result['buyer']['inn']}")
        print(f"КПП: {result['buyer']['kpp']}")
        print(f"Адрес: {result['buyer']['address']}")

        print(f"\n--- Финансы ---")
        print(f"Общая сумма: {result['total_amount']}")
        print(f"Сумма НДС: {result['vat_amount']}")
        print(f"Ставка НДС: {result['vat_rate']}")

        print(f"\n--- Позиции ({len(result['items'])} шт.) ---")
        for i, item in enumerate(result['items'], 1):
            print(f"{i}. {item.get('name', 'N/A')} - {item.get('qty', 'N/A')} шт. - {item.get('price', 'N/A')} руб.")

    except Exception as e:
        print(f"Ошибка при обработке: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Невозможно продолжить: файл не найден")


 Демонстрация работы
Обработка PDF-документа...

  JSON-формат
{
  "document_type": "Счёт на оплату",
  "document_number": "315",
  "document_date": "18.09.2025",
  "supplier": {
    "name": "Банк получателя",
    "inn": "666200126790",
    "kpp": null,
    "address": "620138, Свердловская обл, г Екатеринбург, ул Анны Бычковой, д.\n16",
    "bik": "046577674",
    "account": null,
    "correspondent_account": null
  },
  "buyer": {
    "name": "ПокупательАО \"МК \"ВЫСОТА\"",
    "inn": "6672212600",
    "kpp": "668501001",
    "address": null
  },
  "total_amount": "1659649.00",
  "vat_amount": "1659649.00",
  "vat_rate": "20%",
  "items": [
    {
      "name": "Полуавтоматическая машина для шелкографии с плоской\nкроватью YX3250, линия по производству печатных плат со\nсветодиодной подсветкой SMT, машина для печати паяльной\nпасты",
      "qty": "1",
      "unit": "шт",
      "price": "269 200,00",
      "total": "269 200,00"
    },
    {
      "name": "В наличии в RU Высокоскоростна